# Introduction

Monoids, Monads는 스칼라와 함수적 프로그래밍 세계에서는 반복되어 나오는 테마다. 이장은 Monoid의 구상에 기반한 디자인패턴을 설명한다. 순수 함수적 프로그래밍에서 monoid는 객체 집합에서 계산을 나타내는 구조이다.

Monoid 패턴은 monad 구조 이면의 구상을 가져다가 컨테이너의 멤버에 함수를 적용하는 Monad의 생성과 조합을 다루는 컨테어너 타입을 어떻게 생성하는지 정의하고 다중의 컨테이너가 어떻게 단일 컨테이너를 펴지는지를 정의한다.

Monad 패턴은 다음처럼 함께 그룹질 수 있는 연산의 집합을 제공하는 T 타입의 컨테이너를 식별한다.

Functor 같은 연산들
* monad에서 함수를 적용함에 아무런 영향을 주지 않는 유일한 항등원을 소유한다.
* 동일 타입의 신규 컨테이너를 반환하는 컨테이너의 모든 객체에 함수를 적용할 수 있는 능력
* 함수적 조합 적용력. 2개 함수를 함께 조합한 후 조합된 함수를 적용하면 첫번째 함수를 적용하고 연이어 두번째 함수를 적용한 것과 동일한 효과를 얻는다.

Applicative Functor 같은 연산들
* pure 연산. 컨텍스트 내에 객체를 담는다.
* apply 연산. 이미 있는 컨텍스트에서 하나 이상의 함수들을 취하여 객체들에 적용한다. 생성된 결과는 역시 그 컨텍스트 내에 있다.

Monoid 같은 연산들
* 추가적으로 함께 집합의 요소들을 조합하는 조합 연산
* 다중의 집합을 취해서 단일 집합에 조합하는 연산 (Join이라고 알려졌다)

In [2]:
trait Functor[T[_], A] {
    def fmap(func: A=>A): Functor[T, A]
    val identity: T[A]
}

defined trait Functor

In [3]:
trait ApplicativeFunctor[T[_], A] extends Functor[T, A] {
    def apply(functions: T[A=>A]): ApplicativeFunctor[T, A]
}

defined trait ApplicativeFunctor

<img src='./figures/Fig.39.1 Types used in the Monad example.png'>

In [4]:
trait Monoid[T[_], A] {
    def append(values: T[A]): Monoid[T, A]
}

defined trait Monoid

In [5]:
trait Monad[T[_], A] extends Monoid[T, A] with ApplicativeFunctor[T, A]

defined trait Monad

In [6]:
case class ListMonad[A](val list: List[A]) extends Monad[List, A] {
    val identity = Nil
    
    def fmap(f: A=>A): ListMonad[A] = ListMonad(list.map(f))
    def apply(tf: List[A=>A]): ListMonad[A] = {
        var l: List[A] = list
        tf.foreach(f => l = l.map(f))
        return ListMonad(l)
    }
    def append(l: List[A]) = ListMonad(list ++ l)
}

defined class ListMonad

In [8]:
import scala.collection.mutable.ListBuffer

object ListMonad {
    def join[A](xs: List[ListMonad[A]]): ListMonad[A] = {
        val buffer = ListBuffer[A]()
        xs.foldLeft(buffer)((x, y) => x ++= y.list)
        ListMonad(buffer.toList)
    }
    def pure[A](a: A) = ListMonad(List(a))
}

: 

In [ ]:
val monad = ListMonad[Int](List(1,2))
println(monad)
println(monad append List(6,7))
println(monad fmpa(_+1))

val increase = (x: Int) => x+1
val double = (x: Int) => x*2
println(monad apply(List(increase)))
println(monad apply(List(increase, double)))

val l1 = List(ListMonad(List(1,2)), ListMonad(List(5,6)))
println(ListMonad join l1)
println(ListMonad pure 4)